In [1]:
from my_import import *
df_train = pd.read_csv('df_train.csv')
df_val = pd.read_csv('df_val.csv')
df_test = pd.read_csv('df_test.csv')

#Make sure the genre collumns is in lists not strings
#NEED TO DO THIS EVERYTIME EXPORT DATASET
df_train['genres'] = df_train['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_val['genres'] = df_val['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_test['genres'] = df_test['genres'].apply(lambda x: list(ast.literal_eval(x)))

c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Generating prior probability thresholds
# Flatten all genre lists into one long list
all_genres = sorted(set(genre for sublist in df_train["genres"] for genre in sublist))

# Flatten the genre lists
all_genres_comb = [genre for genre_list in df_train['genres'] for genre in genre_list]

# Count each genre
genre_counts = Counter(all_genres_comb)

# Total number of samples
total_samples = len(df_train)

# Compute prior probability for each genre
genre_priors = {genre: count / total_samples for genre, count in genre_counts.items()}

# Display
prior_threshold=[]
for genre, prob in sorted(genre_priors.items()):
    print(f"{genre}: {prob:.4f}")
    prior_threshold.append(prob)
prior_threshold

Action: 0.3128
Adventure: 0.2180
Comedy: 0.4091
Drama: 0.2069
Fantasy: 0.2368
Historical: 0.0743
Kids: 0.0834
Mecha: 0.0763
Music: 0.0514
Mystery: 0.0666
Romance: 0.1678
School: 0.1360
Sci-Fi: 0.1993
Seinen: 0.0712
Shoujo: 0.0565
Shounen: 0.1603
Slice of Life: 0.1410
Sports: 0.0507
Super Power: 0.0500
Supernatural: 0.1277


[0.3127808988764045,
 0.21797752808988763,
 0.40912921348314607,
 0.20688202247191012,
 0.23679775280898877,
 0.07429775280898876,
 0.08342696629213484,
 0.07626404494382022,
 0.05140449438202247,
 0.06657303370786517,
 0.1678370786516854,
 0.13595505617977527,
 0.19929775280898876,
 0.07120786516853933,
 0.056460674157303374,
 0.16025280898876404,
 0.14101123595505619,
 0.050702247191011234,
 0.05,
 0.12766853932584268]

In [14]:
from sklearn.metrics import confusion_matrix, matthews_corrcoef

def optimize_thresholds_with_precision_constraint(probs, labels, thresholds=np.linspace(0, 1, 1000), min_precision=0.5):
    best_thresholds = []
    for i in range(probs.shape[1]):
        best_f1 = 0
        best_thresh = 0.5
        for t in thresholds:
            pred = (probs[:, i] >= t).astype(int)
            precision = precision_score(labels[:, i], pred, zero_division=0)
            f1 = f1_score(labels[:, i], pred, zero_division=0)
            if f1 > best_f1 and precision >= min_precision:
                best_f1 = f1
                best_thresh = t
        best_thresholds.append(best_thresh)
        print(f"Label {i}: Best threshold = {best_thresh:.2f}, F1 = {best_f1:.4f}")
    return (best_thresholds)

def compute_tpr_tnr(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    tnr = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    return tpr, tnr

def optimize_thresholds_with_mcc_constraint(probs, labels, thresholds=np.linspace(0, 1, 1000)):
    best_thresholds = []

    for i in range(probs.shape[1]):
        best_mcc = -1
        best_thresh = 0.5

        for t in thresholds:
            pred = (probs[:, i] >= t).astype(int)
            tpr, tnr = compute_tpr_tnr(labels[:, i], pred)
            mcc = matthews_corrcoef(labels[:, i], pred)
            if mcc > best_mcc:
                best_mcc = mcc
                best_thresh = t
 

        best_thresholds.append(best_thresh)
        print(f"Label {i}: Best threshold = {best_thresh:.2f}, MCC = {best_mcc:.4f}")
    
    return best_thresholds

optimised_thresh=optimize_thresholds_with_precision_constraint(val_prob,val_labels)
#optimised_thresh=optimize_thresholds_with_mcc_constraint(val_prob,val_labels)

Label 0: Best threshold = 0.28, F1 = 0.6743
Label 1: Best threshold = 0.49, F1 = 0.5393
Label 2: Best threshold = 0.09, F1 = 0.6554
Label 3: Best threshold = 0.99, F1 = 0.3737
Label 4: Best threshold = 0.27, F1 = 0.5912
Label 5: Best threshold = 0.68, F1 = 0.5082
Label 6: Best threshold = 0.22, F1 = 0.5401
Label 7: Best threshold = 1.00, F1 = 0.4480
Label 8: Best threshold = 0.34, F1 = 0.4054
Label 9: Best threshold = 1.00, F1 = 0.0645
Label 10: Best threshold = 0.99, F1 = 0.4496
Label 11: Best threshold = 0.91, F1 = 0.5882
Label 12: Best threshold = 0.09, F1 = 0.5819
Label 13: Best threshold = 1.00, F1 = 0.1667
Label 14: Best threshold = 0.99, F1 = 0.2857
Label 15: Best threshold = 0.99, F1 = 0.3285
Label 16: Best threshold = 0.98, F1 = 0.4135
Label 17: Best threshold = 0.03, F1 = 0.6237
Label 18: Best threshold = 0.50, F1 = 0.0000
Label 19: Best threshold = 1.00, F1 = 0.4532


In [3]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)

df_train['clean_synopsis'] = df_train['synopsis'].apply(clean_text)
df_val['clean_synopsis'] = df_val['synopsis'].apply(clean_text)
df_test['clean_synopsis'] = df_test['synopsis'].apply(clean_text)

#Parameters that can be changed:
#Number of wordds in vocublary
#Class weighting


# from sklearn.feature_extraction.text import CountVectorizer
mlb=MultiLabelBinarizer()
y_train = mlb.fit_transform(df_train['genres'])
#display(y)
vectorizer = CountVectorizer(stop_words="english",max_features=10000,ngram_range=(1,4),binary=True,min_df=2,max_df=0.8)
X_train = vectorizer.fit_transform(df_train['clean_synopsis'])
#display(X_train)
#print(vectorizer.get_feature_names_out(),len(vectorizer.get_feature_names_out()))
#print(X_train.toarray())

y_val= mlb.transform(df_val['genres'])
X_val = vectorizer.transform(df_val['clean_synopsis'])
y_test= mlb.transform(df_test['genres'])
X_test = vectorizer.transform(df_test['clean_synopsis'])

from sklearn.naive_bayes import BernoulliNB

classifier = OneVsRestClassifier(BernoulliNB())#Already tried with balanced class but it is worse
classifier.fit(X_train, y_train)  

OneVsRestClassifier(estimator=BernoulliNB())

In [12]:
# Loop through each class within the OneVsRestClassifier
for class_index, estimator in enumerate(classifier.estimators_):
    print(f"Class '{mlb.classes_[class_index]}' word frequency:")
    
    # Get the feature names from the CountVectorizer
    feature_names = vectorizer.get_feature_names_out()

    # Sum word counts for this specific class
    class_counts = X_train[y_train[:, class_index] == 1].sum(axis=0).A1

    # Combine words with their class-specific frequencies
    word_freq_pairs = list(zip(feature_names, class_counts))

    # Sort words by frequency in descending order
    sorted_words = sorted(word_freq_pairs, key=lambda x: x[1], reverse=True)

    # Display the top 10 words with the highest frequencies
    print("Top 10 words with highest frequency:")
    for word, count in sorted_words[:10]:
        print(f"{word}: {count}")
        
    print("\n" + "=" * 50 + "\n")


Class 'Action' word frequency:
Top 10 words with highest frequency:
world: 662
life: 465
new: 457
year: 452
mysterious: 369
power: 363
friend: 356
battle: 329
day: 316
young: 313


Class 'Adventure' word frequency:
Top 10 words with highest frequency:
world: 492
life: 308
new: 305
friend: 295
year: 293
young: 244
mysterious: 242
power: 237
time: 224
day: 215


Class 'Comedy' word frequency:
Top 10 words with highest frequency:
school: 649
girl: 627
life: 611
new: 583
world: 526
friend: 524
day: 482
time: 411
year: 368
high: 367


Class 'Drama' word frequency:
Top 10 words with highest frequency:
life: 428
year: 343
world: 326
girl: 324
school: 315
friend: 305
new: 305
day: 293
time: 273
young: 251


Class 'Fantasy' word frequency:
Top 10 words with highest frequency:
world: 580
life: 363
new: 321
girl: 313
friend: 300
power: 280
year: 280
day: 271
mysterious: 269
story: 252


Class 'Historical' word frequency:
Top 10 words with highest frequency:
life: 128
young: 114
story: 112
time: 1

In [10]:
# Loop through each class within the OneVsRestClassifier
for class_index, estimator in enumerate(classifier.estimators_):
    print(f"Class '{mlb.classes_[class_index]}' word frequency and probability:")

    # Get the feature names from the CountVectorizer
    feature_names = vectorizer.get_feature_names_out()

    # Sum word counts for this specific class (already calculated previously)
    class_counts = X_train[y_train[:, class_index] == 1].sum(axis=0).A1

    # Extract the log probabilities from the Naive Bayes model
    log_prob = estimator.feature_log_prob_

    # Convert log probabilities to normal probabilities
    prob = np.exp(log_prob)

    # Combine words with their counts and probabilities
    word_stats = list(zip(feature_names, class_counts, prob[0]))

    # Sort words by frequency in descending order
    sorted_words = sorted(word_stats, key=lambda x: x[1], reverse=True)

    # Display the top 10 words with the highest frequencies and their probabilities
    print("Top 10 words with highest frequency and probability:")
    print(f"{'Word':<20} {'Frequency':<10} {'Probability':<15}")
    print("=" * 50)
    for word, count, probability in sorted_words[:10]:
        print(f"{word:<20} {count:<10} {probability:.4f}")
        
    print("\n" + "=" * 50 + "\n")


Class 'Action' word frequency and probability:
Top 10 words with highest frequency and probability:
Word                 Frequency  Probability    
world                662        0.1741
life                 465        0.2172
new                  457        0.1820
year                 452        0.1418
mysterious           369        0.0858
power                363        0.0699
friend               356        0.1712
battle               329        0.0380
day                  316        0.1714
young                313        0.1095


Class 'Adventure' word frequency and probability:
Top 10 words with highest frequency and probability:
Word                 Frequency  Probability    
world                492        0.1835
life                 308        0.2190
new                  305        0.1873
friend               295        0.1614
year                 293        0.1531
young                244        0.1086
mysterious           242        0.0982
power                237        0.08

In [21]:
import pandas as pd
import numpy as np

# Initialize the DataFrame with column names
word_range=10
top_words_df = pd.DataFrame(columns=[f"Top Word {i+1}" for i in range(word_range)])

# Iterate through each class and its corresponding estimator
for class_index, estimator in enumerate(classifier.estimators_):
    genre_name = mlb.classes_[class_index]
    
    # Get the feature names from the vectorizer
    feature_names = vectorizer.get_feature_names_out()

    # Sum word counts for this specific class
    class_counts = X_train[y_train[:, class_index] == 1].sum(axis=0).A1

    # Extract the log probabilities from the Naive Bayes model
    log_prob = estimator.feature_log_prob_

    # Convert log probabilities to normal probabilities
    prob = np.exp(log_prob)

    # Combine words with their counts and probabilities
    word_stats = list(zip(feature_names, class_counts, prob[0]))

    # Sort words by frequency in descending order
    sorted_words = sorted(word_stats, key=lambda x: x[1], reverse=True)

    # Select the top 5 words
    top_5_words = [f"{word} ({probability:.4f})" for word, count, probability in sorted_words[:word_range]]

    # Add the top words to the DataFrame
    top_words_df.loc[genre_name] = top_5_words

# Display the DataFrame with genres as rows and top words as columns
display(top_words_df)


,Top Word 1,Top Word 2,Top Word 3,Top Word 4,Top Word 5,Top Word 6,Top Word 7,Top Word 8,Top Word 9,Top Word 10
Action,world (0.1741),life (0.2172),new (0.1820),year (0.1418),mysterious (0.0858),power (0.0699),friend (0.1712),battle (0.0380),day (0.1714),young (0.1095)
Adventure,world (0.1835),life (0.2190),new (0.1873),friend (0.1614),year (0.1531),young (0.1086),mysterious (0.0982),power (0.0840),time (0.1404),day (0.1688)
Comedy,school (0.1580),girl (0.1858),life (0.2179),new (0.1818),world (0.2347),friend (0.1592),day (0.1599),time (0.1414),year (0.1848),high (0.1029)
Drama,life (0.1947),year (0.1421),world (0.2103),girl (0.1921),school (0.1768),friend (0.1574),new (0.1846),day (0.1526),time (0.1298),young (0.1059)
Fantasy,world (0.1718),life (0.2143),new (0.1889),girl (0.2016),friend (0.1645),power (0.0782),year (0.1593),day (0.1626),mysterious (0.0957),story (0.1462)
Historical,life (0.2123),young (0.1115),story (0.1418),time (0.1364),year (0.1580),world (0.2140),japan (0.0623),man (0.0672),war (0.0557),day (0.1637)
Kids,friend (0.1650),world (0.2154),story (0.1469),new (0.1936),time (0.1423),life (0.2226),child (0.0469),series (0.0875),anime (0.0594),help (0.0875)
Mecha,year (0.1490),earth (0.0517),world (0.2073),new (0.1851),robot (0.0131),pilot (0.0087),war (0.0520),life (0.2172),mysterious (0.1049),battle (0.0634)
Music,music (0.0068),new (0.1871),girl (0.1963),school (0.1846),idol (0.0107),story (0.1461),world (0.2157),day (0.1634),life (0.2186),year (0.1621)
Mystery,world (0.2083),life (0.2107),mysterious (0.1032),begin (0.0975),day (0.1599),friend (0.1668),school (0.1856),detective (0.0048),new (0.1909),girl (0.2004)


In [38]:
import pandas as pd
import numpy as np

# Initialize the DataFrame with column names
influential_words_df = pd.DataFrame(columns=[f"Top Word {i+1}" for i in range(10)])

for class_index, estimator in enumerate(classifier.estimators_):
    genre_name = mlb.classes_[class_index]
    
    # Get feature names and log probabilities
    feature_names = vectorizer.get_feature_names_out()
    log_prob = estimator.feature_log_prob_

    # Calculate log odds (difference between current class and the average of others)
    log_odds = log_prob[0] - np.mean(log_prob, axis=0)

    # Combine words with their log-odds
    word_stats = list(zip(feature_names, log_odds))

    # Sort words by absolute log-odds in descending order
    sorted_words = sorted(word_stats, key=lambda x: abs(x[1]), reverse=True)

    # Select the top 5 most influential words
    top_5_words = [f"{word}({log_odds:.4f}) " for word, log_odds in sorted_words[:10]]#({log_odds:.4f})

    # Add the top words to the DataFrame as a new row
    influential_words_df.loc[genre_name] = pd.Series(top_5_words, index=influential_words_df.columns)

# Display the DataFrame
display(influential_words_df)

#influential_words_df.to_csv('BNB words.csv',  index=False)


,Top Word 1,Top Word 2,Top Word 3,Top Word 4,Top Word 5,Top Word 6,Top Word 7,Top Word 8,Top Word 9,Top Word 10
Action,gokuu(-1.7796),zenigata(-1.7129),tactical(-1.6758),bloodthirsty(-1.6358),jigen(-1.6358),fifth holy(-1.5923),fifth holy grail(-1.5923),fifth holy grail war(-1.5923),gokuu son(-1.5923),rin toosaka(-1.5923)
Adventure,pokémon(-2.0549),gohan(-1.9208),goemon(-1.8807),jigen(-1.8807),bramble(-1.8372),gokuu son(-1.8372),satoshi friend(-1.8372),team rocket(-1.8372),inverse(-1.7896),lina inverse(-1.7896)
Comedy,hilarious(-1.9165),comedic(-1.8843),yorozuya(-1.5700),gag(-1.5541),doraemon(-1.5032),zenigata(-1.5032),arale(-1.4662),comedy(-1.4662),follows daily(-1.4662),gintoki(-1.4662)
Drama,takashi natsume(-1.8227),uninhabitable(-1.7700),karasuno(-1.7111),karasuno high(-1.7111),karasuno high school(-1.7111),akemi(-1.6444),kircheis(-1.6444),meiko(-1.6444),shinji ikari(-1.6444),siegfried(-1.6444)
Fantasy,precure(-2.1303),pokémon(-2.0014),pokemon(-1.9388),sorceress(-1.9388),luffy(-1.9043),pikachu(-1.8272),vegeta(-1.8272),fifth holy(-1.7837),fifth holy grail(-1.7837),fifth holy grail war(-1.7837)
Historical,hijikata(-2.2992),yorozuya(-2.2670),edo(-2.2465),bakumatsu(-2.1554),demon slayer(-2.1554),genji(-2.1554),gintoki(-2.1554),mushishi(-2.1554),mushishi ginko(-2.1554),shogunate(-2.0642)
Kids,pikachu(-2.6557),pokémon(-2.6134),pokemon(-2.5508),safety anime(-2.3958),satoshi friend(-2.3958),team rocket(-2.3958),satoshi pikachu(-2.2954),legendary pokémon(-2.2365),backkom(-2.1698),educational anime(-2.1698)
Mecha,mobile suit(-2.9115),zeon(-2.7909),gundam(-2.7556),universal century(-2.6317),aznable(-2.4444),char aznable(-2.4444),robot known(-2.4444),earth federation(-2.4211),principality zeon(-2.3967),super robot(-2.3967)
Music,shower tv(-2.5537),space shower(-2.5537),space shower tv(-2.5537),id directed(-2.4948),shower tv station(-2.4948),shower tv station id(-2.4948),space shower tv station(-2.4948),station id directed(-2.4948),tv station id directed(-2.4948),ayanagi(-2.3509)
Mystery,edogawa(-2.9074),conan edogawa(-2.8406),kogorou mouri(-2.6724),detective boy(-2.6379),ryougi(-2.5608),shiki ryougi(-2.5173),mikiya(-2.4696),mikiya kokutou(-2.4696),kindaichi(-2.4169),mouri(-2.4169)


In [29]:
import pandas as pd
import numpy as np

# Initialize the DataFrame for the most positively impactful words
positive_words_df = pd.DataFrame(columns=[f"Top Word {i+1}" for i in range(5)])

# Iterate through each class and its corresponding Naive Bayes estimator
for class_index, estimator in enumerate(classifier.estimators_):
    genre_name = mlb.classes_[class_index]
    
    # Get feature names from the vectorizer
    feature_names = vectorizer.get_feature_names_out()

    # Extract the log probabilities from the Naive Bayes model
    log_prob = estimator.feature_log_prob_

    # Convert log probabilities to normal probabilities
    prob = np.exp(log_prob[0])  # Take the first row since it represents the current class

    # Combine words with their probabilities
    word_stats = list(zip(feature_names, prob))

    # Sort words by probability in descending order (to get the most positive impact)
    sorted_words = sorted(word_stats, key=lambda x: x[1], reverse=True)

    # Select the top 5 most positively impactful words
    top_5_words = [f"{word} ({probability:.4f})" for word, probability in sorted_words[:5]]

    # Add the top words to the DataFrame as a new row
    positive_words_df.loc[genre_name] = pd.Series(top_5_words, index=positive_words_df.columns)

# Display the DataFrame
display(positive_words_df)


,Top Word 1,Top Word 2,Top Word 3,Top Word 4,Top Word 5
Action,girl (0.2261),life (0.2172),school (0.2112),new (0.1820),world (0.1741)
Adventure,life (0.2190),school (0.2190),girl (0.2162),new (0.1873),world (0.1835)
Comedy,world (0.2347),life (0.2179),girl (0.1858),year (0.1848),new (0.1818)
Drama,world (0.2103),life (0.1947),girl (0.1921),new (0.1846),school (0.1768)
Fantasy,life (0.2143),school (0.2090),girl (0.2016),new (0.1889),world (0.1718)
Historical,world (0.2140),life (0.2123),girl (0.2029),school (0.1937),new (0.1929)
Kids,life (0.2226),world (0.2154),girl (0.2076),school (0.1953),new (0.1936)
Mecha,life (0.2172),world (0.2073),girl (0.2022),school (0.1914),new (0.1851)
Music,life (0.2186),world (0.2157),girl (0.1963),new (0.1871),school (0.1846)
Mystery,life (0.2107),world (0.2083),girl (0.2004),new (0.1909),school (0.1856)


In [4]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, hamming_loss, jaccard_score, roc_auc_score
import numpy as np
import pandas as pd

def predict_genres_with_naives(classifier, vectorizer, df_source, mlb, id2label, threshold=0.5):
    # Vectorize clean synopsis
    X = vectorizer.transform(df_source['clean_synopsis'])
    y_true = mlb.transform(df_source['genres'])

    # Predict probabilities and apply threshold
    probabilities = classifier.predict_proba(X)
    probabilities = np.array(probabilities)  # list of arrays → array
    if isinstance(probabilities, list):  # OneVsRestClassifier returns list of arrays
        probabilities = np.vstack([p[:, 1] for p in probabilities]).T
    predictions = (probabilities >= threshold).astype(int)

    # Decode predicted genres using id2label
    predicted_genres = []
    for row in predictions:
        genres = [id2label[i] for i, val in enumerate(row) if val == 1]
        predicted_genres.append(genres)

    # === Calculate metrics ===
    f1 = f1_score(y_true, predictions, average='macro')
    precision = precision_score(y_true, predictions, average='macro')
    recall = recall_score(y_true, predictions, average='macro')
    accuracy = accuracy_score(y_true, predictions)
    hamming = hamming_loss(y_true, predictions)
    jaccard = jaccard_score(y_true, predictions, average='macro')
    hit_rate = (np.logical_and(y_true, predictions).sum(axis=1) > 0).mean()
    try:
        roc_auc = roc_auc_score(y_true, probabilities, average='macro')
    except ValueError:
        roc_auc = np.nan

    metrics_df = pd.DataFrame({
        'Metric': [
            'F1 Score', 'Precision', 'Recall', 'Exact Match Accuracy',
            'Hamming Loss', 'Jaccard Score', 'Hit Rate', 'ROC AUC'
        ],
        'Value': [
            f1, precision, recall, accuracy,
            hamming, jaccard, hit_rate, roc_auc
        ]
    })

    # Build result DataFrame
    result_df = pd.DataFrame({
        "synopsis": df_source["synopsis"].values,
        "true_genres": df_source["genres"].values,
        "predicted_genres": predicted_genres
    })

    print("\n📊 Evaluation Metrics:")
    print(metrics_df)

    return result_df, metrics_df, probabilities, y_true, predictions

use_thresh=0.5
id2label = {i: label for i, label in enumerate(mlb.classes_)}
df_train_results, train_metrics, train_prob, train_labels, train_pred  = predict_genres_with_naives(classifier, vectorizer, df_train, mlb, id2label, threshold=use_thresh)
df_val_results, val_metrics, val_prob, val_labels, val_pred = predict_genres_with_naives(classifier, vectorizer, df_val, mlb, id2label, threshold=use_thresh)
df_test_results, test_metrics, test_prob, test_labels, test_pred = predict_genres_with_naives(classifier, vectorizer, df_test, mlb, id2label, threshold=use_thresh)


📊 Evaluation Metrics:
                 Metric     Value
0              F1 Score  0.610425
1             Precision  0.595912
2                Recall  0.632252
3  Exact Match Accuracy  0.160534
4          Hamming Loss  0.104115
5         Jaccard Score  0.445571
6              Hit Rate  0.817275
7               ROC AUC  0.899961

📊 Evaluation Metrics:
                 Metric     Value
0              F1 Score  0.458203
1             Precision  0.453853
2                Recall  0.479874
3  Exact Match Accuracy  0.056054
4          Hamming Loss  0.152018
5         Jaccard Score  0.305168
6              Hit Rate  0.762332
7               ROC AUC  0.811003

📊 Evaluation Metrics:
                 Metric     Value
0              F1 Score  0.437662
1             Precision  0.433317
2                Recall  0.457388
3  Exact Match Accuracy  0.041526
4          Hamming Loss  0.153311
5         Jaccard Score  0.290723
6              Hit Rate  0.735129
7               ROC AUC  0.807554


In [7]:

from sklearn.metrics import classification_report

print(classification_report(train_labels, train_pred, target_names=all_genres))
print(classification_report(val_labels, val_pred, target_names=all_genres))
print(classification_report(test_labels, test_pred, target_names=all_genres))

               precision    recall  f1-score   support

       Action       0.71      0.70      0.71      2227
    Adventure       0.65      0.74      0.69      1552
       Comedy       0.74      0.86      0.79      2913
        Drama       0.55      0.61      0.58      1473
      Fantasy       0.71      0.69      0.70      1686
   Historical       0.57      0.58      0.57       529
         Kids       0.69      0.72      0.70       594
        Mecha       0.52      0.63      0.57       543
        Music       0.66      0.46      0.55       366
      Mystery       0.53      0.63      0.57       474
      Romance       0.56      0.64      0.60      1195
       School       0.60      0.71      0.65       968
       Sci-Fi       0.73      0.70      0.72      1419
       Seinen       0.47      0.53      0.50       507
       Shoujo       0.43      0.45      0.44       402
      Shounen       0.62      0.61      0.62      1141
Slice of Life       0.57      0.68      0.62      1004
       Sp

c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [8]:
display(df_train_results)
display(df_val_results)
display(df_test_results)

,synopsis,true_genres,predicted_genres
0,Shuramaru is hated and feared by the villagers...,[Supernatural],[]
1,"Sometime in the future, the world was complete...","[Action, Adventure]",[]
2,"Set in 2014, the anime follows the adventures ...",[Comedy],[Comedy]
3,"This story is about Mick, a sleeping chironomi...",[Adventure],"[Action, Adventure, Sci-Fi]"
4,The anime is based on MegaHouse's line of Zigu...,"[Action, Mecha]","[Action, Comedy]"
...,...,...,...
7115,Picture Drama episodes included in each DVD vo...,"[Action, Adventure, Comedy, Drama, Fantasy, Ro...",[Comedy]
7116,"On a hot summer day, Takashi Natsume and his f...","[Drama, Shoujo, Slice of Life, Supernatural]","[Drama, Mystery, Shoujo, Slice of Life, Supern..."
7117,"Due to the arrival of aliens called the ""Amant...","[Action, Comedy, Historical, Mecha, Sci-Fi, Sh...","[Action, Comedy, Historical, Mecha, Sci-Fi, Sh..."
7118,The Konohagakure Grand Sports Festival has beg...,"[Action, Comedy, Fantasy, Shounen, Sports]","[Action, Adventure, Comedy, Shounen, Sports, S..."


,synopsis,true_genres,predicted_genres
0,Follows a pig whose family's mission is to col...,"[Comedy, Kids]",[Comedy]
1,The story begins 15 years after the demon king...,"[Comedy, Fantasy, Slice of Life]",[Fantasy]
2,"The story centers on Otarou, a creature themed...",[Comedy],[Comedy]
3,follows the adventures of an adorable jet plan...,"[Adventure, Kids]","[Adventure, Kids]"
4,"An elite soldier, whose nickname is God of Dea...","[Action, Adventure, Comedy, Romance, Sci-Fi]",[Comedy]
...,...,...,...
887,This special is showing a day of Eclair's life...,"[Fantasy, Shounen]",[Comedy]
888,Episode 0 of . Kanami and Mihono are self-awar...,[Action],[Comedy]
889,The ad's crossover theme matches a tote bag bu...,"[Seinen, Slice of Life]",[Comedy]
890,A girl's classmate disappears during a game of...,"[Shoujo, Supernatural]",[Comedy]


,synopsis,true_genres,predicted_genres
0,The daily life of the Motsumoto family. The th...,"[Comedy, Slice of Life]",[Comedy]
1,"The Soul Tree, the great source of our race. ...","[Action, Adventure, Kids]","[Action, Adventure, Fantasy]"
2,"Fairies living in a fluffy forest, where both ...","[Fantasy, Kids]",[Comedy]
3,A cyborg warrior from an ancient Antarctic kin...,"[Action, Mecha, Sci-Fi]","[Action, Adventure, Sci-Fi]"
4,The 2018 LINE sticker set Poccolies is inspiri...,"[Kids, Slice of Life]",[Comedy]
...,...,...,...
886,A series of four animated prequels to Street F...,[Action],[]
887,"According to HMV, the DVD & Blu-ray volume 7 o...","[Comedy, Seinen, Sports]",[Comedy]
888,"Lucius ends up in a space station, where he ex...","[Comedy, Seinen]",[]
889,"The ""Megumi and Taiyou"" commercial depicts the...","[Romance, Slice of Life]",[]


In [5]:
roc_auc_per_genre = roc_auc_score(test_labels, test_prob, average=None)
for label, auc in zip(all_genres, roc_auc_per_genre):
    print(f"{label}: ROC AUC = {auc:.4f}")


Action: ROC AUC = 0.8623
Adventure: ROC AUC = 0.8111
Comedy: ROC AUC = 0.7549
Drama: ROC AUC = 0.7242
Fantasy: ROC AUC = 0.8507
Historical: ROC AUC = 0.8321
Kids: ROC AUC = 0.8596
Mecha: ROC AUC = 0.8644
Music: ROC AUC = 0.8569
Mystery: ROC AUC = 0.7988
Romance: ROC AUC = 0.7750
School: ROC AUC = 0.8657
Sci-Fi: ROC AUC = 0.8609
Seinen: ROC AUC = 0.6429
Shoujo: ROC AUC = 0.7723
Shounen: ROC AUC = 0.7217
Slice of Life: ROC AUC = 0.8485
Sports: ROC AUC = 0.8806
Super Power: ROC AUC = 0.7718
Supernatural: ROC AUC = 0.7965


In [9]:
df=pd.DataFrame(roc_auc_per_genre)
display(df)
df.to_csv("BNBROC.csv")

,0
0,0.862329
1,0.811136
2,0.754864
3,0.724202
4,0.850687
5,0.832084
6,0.859638
7,0.864444
8,0.856933
9,0.798831


In [ ]:
from sklearn.metrics import matthews_corrcoef
import numpy as np
import pandas as pd

def mcc_per_label(y_true, y_pred, label_names=None):
    n_labels = y_true.shape[1]
    mcc_scores = []

    for i in range(n_labels):
        mcc = matthews_corrcoef(y_true[:, i], y_pred[:, i])
        mcc_scores.append(mcc)

    if label_names is None:
        label_names = [f"Label_{i}" for i in range(n_labels)]

    return pd.Series(mcc_scores, index=label_names, name="MCC")

MCC_labels=mcc_per_label(val_labels,val_pred,label_names=all_genres)
display(MCC_labels)

In [3]:
#Parameters that can be changed:
#Number of wordds in vocublary
#Class weighting


# from sklearn.feature_extraction.text import CountVectorizer
mlb=MultiLabelBinarizer()
y_train = mlb.fit_transform(df_train['genres'])
#display(y)
vectorizer = TfidfVectorizer(stop_words="english",max_features=10000,ngram_range=(1,4),binary=True)
X_train = vectorizer.fit_transform(df_train['clean_synopsis'])
#display(X_train)
#print(vectorizer.get_feature_names_out(),len(vectorizer.get_feature_names_out()))
#print(X_train.toarray())

y_val= mlb.transform(df_val['genres'])
X_val = vectorizer.transform(df_val['clean_synopsis'])
y_test= mlb.transform(df_test['genres'])
X_test = vectorizer.transform(df_test['clean_synopsis'])

from sklearn.naive_bayes import BernoulliNB

classifier = OneVsRestClassifier(BernoulliNB())#Already tried with balanced class but it is worse
classifier.fit(X_train, y_train)                                    #At least when it is not balanced it can overfit the train class

# from sklearn.metrics import classification_report
#samples	How well did we predict each movie's genres?	✅ Best for your task
#micro	    Overall genre prediction accuracy	            ✅ Good for global view
#macro	    Equal focus on rare and frequent genres	        ⚠️ Careful with imbalance
#weighted	Genre performance balanced by frequency	        ✅ Fair but skew-sensitive
#USE SAMPLES


print("Report for Train dataset")
threshold=0.2
y_pred_train=LR_OVA_prob(X_train,threshold,classifier)
print_report(y_train,y_pred_train,"samples avg","f1-score",mlb)
jaccard = jaccard_score(y_train, y_pred_train, average='samples')
print("Jaccard Similarity:", jaccard)
hr = hit_rate(y_train, y_pred_train)
print("Hit Rate:", hr)

print("----------------------")
print("Report for Val dataset")
y_pred_val=LR_OVA_prob(X_val,threshold,classifier)
print_report(y_val,y_pred_val,"samples avg","f1-score",mlb)
jaccard = jaccard_score(y_val, y_pred_val, average='samples')
print("Jaccard Similarity:", jaccard)
hr = hit_rate(y_val, y_pred_val)
print("Hit Rate:", hr)

print("----------------------")
print("Report for Test dataset")
y_pred_test=LR_OVA_prob(X_test,threshold,classifier)
print_report(y_test,y_pred_test,"samples avg","f1-score",mlb)
jaccard = jaccard_score(y_test, y_pred_test, average='samples')
print("Jaccard Similarity:", jaccard)
hr = hit_rate(y_val, y_pred_val)
print("Hit Rate:", hr)



display(result_table(y_train,y_pred_train,df_train,mlb))
display(result_table(y_val,y_pred_val,df_val,mlb))
display(result_table(y_test,y_pred_test,df_test,mlb))

Report for Train dataset
samples avg f1-score = 0.5258426441706441
Jaccard Similarity: 0.42231395211411027
Hit Rate: 0.8153143354361541
----------------------
Report for Val dataset
samples avg f1-score = 0.4240106743663475
Jaccard Similarity: 0.31489420044936856
Hit Rate: 0.7848990342405618
----------------------
Report for Test dataset
samples avg f1-score = 0.4270376861002074
Jaccard Similarity: 0.31725283550875344
Hit Rate: 0.7848990342405618


c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\User\anaconda3\envs\nlp_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Synopsis,Actual Genres,Predicted Genres
0,"Sometime in the future, the world was complete...","(Action, Adventure)","(Adventure, Fantasy)"
1,"Set in 2014, the anime follows the adventures ...","(Comedy,)","(Comedy, Slice of Life)"
2,Follows a pig whose family's mission is to col...,"(Comedy, Kids)","(Comedy,)"
3,"In honor of the 2018 World Cup, this season of...","(Kids, Sci-Fi, Sports, Super Power)","(Kids, Sports)"
4,"Fairies living in a fluffy forest, where both ...","(Fantasy, Kids)","(Comedy, Kids)"
...,...,...,...
9189,The Konohagakure Grand Sports Festival has beg...,"(Action, Comedy, Fantasy, Other, Shounen, Sports)","(Action, Adventure, Comedy, Other, Shounen, Sp..."
9190,Special bundled with the Blu-ray/DVD volume of .,"(Ecchi,)","(Comedy,)"
9191,"According to the official Hobby Japan website,...","(Comedy, Ecchi, Fantasy, Parody)","(Comedy,)"
9192,A series of comedic shorts featuring chibi ver...,"(Adventure, Comedy, Fantasy, Parody)","(Comedy,)"


,Synopsis,Actual Genres,Predicted Genres
0,Shuramaru is hated and feared by the villagers...,"(Supernatural,)",()
1,The daily life of the Motsumoto family. The th...,"(Comedy, Slice of Life)","(Comedy,)"
2,"The Soul Tree, the great source of our race. ...","(Action, Adventure, Kids)","(Action, Adventure, Fantasy, Kids, Other)"
3,Himeno Mimika is a daughter of historical cook...,"(Kids, Other, Slice of Life)","(Comedy,)"
4,The third generation of J SOUL BROTHERS from E...,"(Kids, Music)","(Comedy, Kids, Music)"
...,...,...,...
1134,Specials included in the OVA of Hyakujitsu no ...,"(Other, Parody)","(Comedy,)"
1135,DVD/BD specials featuring two female cast memb...,"(Ecchi, Seinen)","(Comedy,)"
1136,"As college entrance examinations draw near, tw...","(School, Slice of Life)","(Comedy, Romance, School, Slice of Life)"
1137,Two special episodes of included on Blu-ray. 1...,"(Ecchi, Slice of Life)","(Comedy,)"


,Synopsis,Actual Genres,Predicted Genres
0,The story begins 15 years after the demon king...,"(Comedy, Fantasy, Slice of Life)","(Adventure, Fantasy)"
1,A cyborg warrior from an ancient Antarctic kin...,"(Action, Mecha, Sci-Fi)","(Action, Adventure, Sci-Fi)"
2,"The story centers on Otarou, a creature themed...","(Comedy,)","(Comedy, Slice of Life)"
3,"An elite soldier, whose nickname is God of Dea...","(Action, Adventure, Comedy, Romance, Sci-Fi)","(Action, Comedy)"
4,"Based on the real life-story of Pink Lady, a f...","(Drama, Music, Slice of Life)","(Comedy,)"
...,...,...,...
1122,An unaired episode released with the sixth BD ...,"(Sci-Fi, Seinen)","(Comedy,)"
1123,Short extras of the anime included in each of ...,"(Romance,)","(Comedy,)"
1124,Short specials included in the Blu-ray release...,"(Sci-Fi, Seinen)","(Comedy,)"
1125,Three episodes aired on TV as part of the Ribo...,"(Shoujo,)","(Comedy, School, Slice of Life)"
